In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import ast

In [27]:
stop = ["el", "la", "los", "las", "a", "al", "ante", "bajo", "cabe", "con", "contra", "de", "del", "desde", "con", "contra", "en", "entre", "hacia", "hasta", "para", "por", "según", "sin", "so", "sobre", "tras"]
def remove_stop(answer):
    for item in stop:
        r = answer.lower().find(item + " ")
        if r == 0:
            answer = answer[len(item)+1:]
    return answer
            

In [28]:
directoryPath = "./sqac/"
filename = "test.json"
js = json.load(open(directoryPath + filename, encoding='utf-8'))
text_complet = js['data']

column_names = ["source", "id", "question", "answer", "answer_no_stop", "answer_start"]
df = pd.DataFrame(columns = column_names)  
for paragraph in text_complet:
    source = paragraph["source"]
    for context in text_complet[text_complet.index(paragraph)]['paragraphs']:
            a =  context['qas']    
            for element in a:
                question = element["question"]
                id = element['id']
                answers = element['answers'] 
                for ans in answers:
                    answer = ans['text']
                    answer_no_stop = remove_stop(answer)                   
                    thestart = ans["answer_start"]
      
                    df.loc[len(df.index)] = [source, id, question, answer, answer_no_stop, thestart] 

# Dataset

In [29]:
df

,source,id,question,answer,answer_no_stop,answer_start
0,wikipedia,56ea5a6b-c494-43fb-8af9-f084e1b3ff11,¿Cuál es el nombre que se le da a la unidad mo...,célula,célula,3
1,wikipedia,82ff0c00-c4a6-4677-af88-b9a05cce3b3f,"¿De dónde proviene el término ""célula"")","del latín cellula, diminutivo de cella, ‘celda’","latín cellula, diminutivo de cella, ‘celda’",11
2,wikipedia,7080811e-67c4-472f-94ce-afa1b69fb4a1,¿Cómo se denominan a los seres vivos con una ú...,unicelulares,unicelulares,328
3,wikipedia,8c5ab52a-3306-4389-ab5c-b5a394fbf77b,¿Dónde se encuentra el material genético en la...,el núcleo,núcleo,274
4,wikipedia,cb1d5d94-ae86-4167-a8df-1de5a8a83038,¿Cómo se llaman las células que necesitan las ...,gliales,gliales,640
...,...,...,...,...,...,...
1905,wikinews,58e72dc4-5c2a-444d-9d45-3db47f74b087,¿Cómo marcó gol Higuaín?,de volea,volea,449
1906,wikinews,726eaa9b-b8b9-4646-8612-fe76cf008b08,¿Cuál es el próximo contrincante de Argentina?,los Países Bajos,Países Bajos,1592
1907,wikinews,eb8e7345-8168-4b09-bc95-6607a32de1c2,¿Quién se ha impuesto en las elecciones para l...,El candidato pro-ruso Victor Yanukovich,candidato pro-ruso Victor Yanukovich,21
1908,wikinews,2dc32963-7057-465c-9c4f-edcef5525f2e,¿Qué porcentaje de apoyo ha logrado Yanukovich...,"48,49%","48,49%",208


# Respuestas del dataset + modelos

In [30]:
ans_column = ["answer"]
match_column = []
models = []

for filename in os.listdir(directoryPath):
    if filename != "test.json":
        print("afegint resultats", filename)
        file = open(directoryPath+filename, "r")
        dic = ast.literal_eval(file.read())
        
        models.append(filename[:-5])
        
        column_name = "answers_" + filename[:-5]
        ans_column.append(column_name)
        df[column_name] = ""
        
        no_stop = "no_stop_" + filename[:-5]
        df[no_stop] = ""
        
        match = "match_" + filename[:-5]
        df[match] = "no"
        match_column.append(match)
        
        for i, key in enumerate(dic):
            if key != df.loc[i, "id"]:
                print("error", filename, key)
                
            df.loc[i, column_name] = dic[key]
            no_stop_answer = remove_stop(dic[key])
            df.loc[i, no_stop] = no_stop_answer
            if dic[key] == df.loc[i, "answer"]:
                df.loc[i, match] = "exact"
            elif no_stop_answer == df.loc[i, "answer_no_stop"]:
                df.loc[i, match] = "no_stop"
            else:
                r = dic[key].find(df.loc[i, "answer"])
                s = df.loc[i, "answer"].find(dic[key])
                if r != -1 or s != -1:
                    df.loc[i, match] = "contains"
                    

afegint resultats beto.json
afegint resultats bne_base_new.json
afegint resultats mbert.json


In [31]:
df[ans_column]

,answer,answers_beto,answers_bne_base_new,answers_mbert
0,célula,La célula,La célula,La célula
1,"del latín cellula, diminutivo de cella, ‘celda’",del latín cellula,del latín cellula,del latín cellula
2,unicelulares,unicelulares,unicelulares,unicelulares
3,el núcleo,el núcleo,el núcleo,el núcleo
4,gliales,células gliales,células gliales,las células gliales
...,...,...,...,...
1905,de volea,remató de volea un balón que había rebotado en...,de volea,remató de volea
1906,los Países Bajos,Bélgica,Países Bajos,Bélgica
1907,El candidato pro-ruso Victor Yanukovich,El candidato pro-ruso Victor Yanukovich derrot...,El candidato pro-ruso Victor Yanukovich,El candidato pro-ruso Victor Yanukovich
1908,"48,49%","el 48,49%","el 48,49%","el 48,49% de los votos, mientras que su rival ..."


# Resultados de cada modelo
* exact: la respuesta es exactamente igual a la del dataset. 
* no stop: la respuesta es igual si no se considera el primero stopword. Ej: _célula_ vs _la célula_
* contains: la respuesta del modelo contiene la respuesta del datset, o a la inversa. Ej _del latín cellula_ vs _del latín cellula, diminutivo de cella, ‘celda’_ 
* no: no hay ninguna coincidencia en las respuestas. Ej. _los Países Bajos_ vs _Bélgica_

In [32]:
for item in match_column:
    print(item)
    print(df[item].value_counts(normalize=True))
    print("")

match_beto
exact       0.604188
contains    0.192147
no          0.153403
no_stop     0.050262
Name: match_beto, dtype: float64

match_bne_base_new
exact       0.580628
contains    0.208377
no          0.150262
no_stop     0.060733
Name: match_bne_base_new, dtype: float64

match_mbert
exact       0.571728
contains    0.204712
no          0.172251
no_stop     0.051309
Name: match_mbert, dtype: float64



# Respuestas del bne_base_new

In [33]:
df[["answer", "answers_bne_base_new", "match_bne_base_new"]]

,answer,answers_bne_base_new,match_bne_base_new
0,célula,La célula,no_stop
1,"del latín cellula, diminutivo de cella, ‘celda’",del latín cellula,contains
2,unicelulares,unicelulares,exact
3,el núcleo,el núcleo,exact
4,gliales,células gliales,contains
...,...,...,...
1905,de volea,de volea,exact
1906,los Países Bajos,Países Bajos,no_stop
1907,El candidato pro-ruso Victor Yanukovich,El candidato pro-ruso Victor Yanukovich,exact
1908,"48,49%","el 48,49%",no_stop


# Fuentes

In [34]:
df["source"].value_counts()

wikipedia    867
ancora       544
wikinews     499
Name: source, dtype: int64

In [35]:
df_wikipedia = df[df['source']=="wikipedia"]
df_ancora = df[df['source']=="ancora"]
df_wikinews = df[df['source']=="wikinews"]
sort_of_match = ["exact", "no_stop", "contains", "no_match"]
new_columns = []
res_wikipedia = []
res_wikinews = []
res_ancora = []
for model in models:
    for sort in sort_of_match:
        new_columns.append(sort + "_" + model)
        column = "match_" + model
    res_wikipedia.append(df_wikipedia[column].value_counts().exact)
    res_wikipedia.append(df_wikipedia[column].value_counts().no_stop)
    res_wikipedia.append(df_wikipedia[column].value_counts().contains)
    res_wikipedia.append(df_wikipedia[column].value_counts().no)
    
    res_wikinews.append(df_wikinews[column].value_counts().exact)
    res_wikinews.append(df_wikinews[column].value_counts().no_stop)
    res_wikinews.append(df_wikinews[column].value_counts().contains)
    res_wikinews.append(df_wikinews[column].value_counts().no)
    
    res_ancora.append(df_ancora[column].value_counts().exact)
    res_ancora.append(df_ancora[column].value_counts().no_stop)
    res_ancora.append(df_ancora[column].value_counts().contains)
    res_ancora.append(df_ancora[column].value_counts().no)
                   
df_results = pd.DataFrame(columns = ["results", "wikipedia", "ancora", "wikinews"])    
df_results["results"] = new_columns   
df_results["wikipedia"] = res_wikipedia
df_results["ancora"] = res_ancora
df_results["wikinews"] = res_wikinews

In [36]:
df_results

,results,wikipedia,ancora,wikinews
0,exact_beto,498,341,315
1,no_stop_beto,45,33,18
2,contains_beto,194,92,81
3,no_match_beto,130,78,85
4,exact_bne_base_new,468,331,310
5,no_stop_bne_base_new,49,45,22
6,contains_bne_base_new,224,87,87
7,no_match_bne_base_new,126,81,80
8,exact_mbert,484,322,286
9,no_stop_mbert,43,34,21


In [37]:
sort_of_match = ["exact", "no_stop", "contains", "no_match"]
new_columns = []
res_wikipedia = []
res_wikinews = []
res_ancora = []
for model in models:
    for sort in sort_of_match:
        new_columns.append(sort + "_" + model)
        column = "match_" + model
    res_wikipedia.append(df_wikipedia[column].value_counts(normalize=True).exact)
    res_wikipedia.append(df_wikipedia[column].value_counts(normalize=True).no_stop)
    res_wikipedia.append(df_wikipedia[column].value_counts(normalize=True).contains)
    res_wikipedia.append(df_wikipedia[column].value_counts(normalize=True).no)
    
    res_wikinews.append(df_wikinews[column].value_counts(normalize=True).exact)
    res_wikinews.append(df_wikinews[column].value_counts(normalize=True).no_stop)
    res_wikinews.append(df_wikinews[column].value_counts(normalize=True).contains)
    res_wikinews.append(df_wikinews[column].value_counts(normalize=True).no)
    
    res_ancora.append(df_ancora[column].value_counts(normalize=True).exact)
    res_ancora.append(df_ancora[column].value_counts(normalize=True).no_stop)
    res_ancora.append(df_ancora[column].value_counts(normalize=True).contains)
    res_ancora.append(df_ancora[column].value_counts(normalize=True).no)

df_percentil = pd.DataFrame(columns = ["results", "wikipedia", "ancora", "wikinews"])    
df_percentil["results"] = new_columns   
df_percentil["wikipedia"] = res_wikipedia
df_percentil["ancora"] = res_ancora
df_percentil["wikinews"] = res_wikinews                   
df_percentil

,results,wikipedia,ancora,wikinews
0,exact_beto,0.574394,0.626838,0.631263
1,no_stop_beto,0.051903,0.060662,0.036072
2,contains_beto,0.223760,0.169118,0.162325
3,no_match_beto,0.149942,0.143382,0.170341
4,exact_bne_base_new,0.539792,0.608456,0.621242
5,no_stop_bne_base_new,0.056517,0.082721,0.044088
6,contains_bne_base_new,0.258362,0.159926,0.174349
7,no_match_bne_base_new,0.145329,0.148897,0.160321
8,exact_mbert,0.558247,0.591912,0.573146
9,no_stop_mbert,0.049596,0.062500,0.042084


# Análisis de las respuestas

In [38]:
df_all_ok = pd.DataFrame(columns = ["id", "question", "answer"]) 
df_alt_ans = pd.DataFrame(columns = ["id", "question", "answer", "alt_answer", "match"]) 
df_1h = pd.DataFrame(columns = ["id", "question", "answer", "bne_base_new", "mbert", "beto", "model_dif"]) 
columns = ["answer_no_stop"]
for model in models:
    columns.append("no_stop_" + model)
    
for k,row in df.iterrows():
    results = []
    r_columns = columns[:] 
    for column in columns:
        r_columns.pop(0)
        if len(r_columns) > 0:
            for r_column in r_columns:
                if df.loc[k, column] == df.loc[k, r_column]:
                    results.append("True")
                else:
                    results.append("False")
        dataset_results = results[:len(models)]
        models_results = results[len(models):]
    if dataset_results.count("True") == len(models):
        df_all_ok.loc[len(df_all_ok.index)] = [df.loc[k, "id"], df.loc[k, "question"], df.loc[k, "answer_no_stop"]]
    elif dataset_results.count("True") == 1:
        for i, r in enumerate(dataset_results):
            if r == "True":
                model_dif = models[i]              
        df_1h.loc[len(df_all_ok.index)] = [df.loc[k, "id"], df.loc[k, "question"], df.loc[k, "answer_no_stop"], df.loc[k, "no_stop_bne_base_new"], df.loc[k, "no_stop_mbert"], df.loc[k, "no_stop_beto"], model_dif]
    elif dataset_results.count("True") == 0:
        if models_results.count("True") == len(models_results):
            df_alt_ans.loc[len(df_all_ok.index)] = [df.loc[k, "id"], df.loc[k, "question"], df.loc[k, "answer_no_stop"], df.loc[k, "no_stop_bne_base_new"], df.loc[k, "match_bne_base_new"]] 
    
        
    

## Preguntas en que todos los modelos han dado la misma respuesta, igual al dataset

In [39]:
df_all_ok

,id,question,answer
0,56ea5a6b-c494-43fb-8af9-f084e1b3ff11,¿Cuál es el nombre que se le da a la unidad mo...,célula
1,7080811e-67c4-472f-94ce-afa1b69fb4a1,¿Cómo se denominan a los seres vivos con una ú...,unicelulares
2,8c5ab52a-3306-4389-ab5c-b5a394fbf77b,¿Dónde se encuentra el material genético en la...,núcleo
3,ad218c32-a803-43db-9b2d-19cf3d969f1d,¿Cuál es el componente primordial de la pared ...,celulosa
4,eeaeee34-405a-4ddd-bbea-330c459c46a6,¿Qué instrumento permitió el estudio de las cé...,microscopio rudimentario de lentes compuestas
...,...,...,...
929,a0f32196-8a01-4c34-b3b7-250c4d6eddb9,¿A qué países viaja Condoleezza Rice?,"Brasil, Colombia, Chile y El Salvador"
930,8b7753c6-c27e-4b13-bb64-8b33710a3c90,¿Qué temas tratará Condoleezza Rice durante su...,"tráfico de drogas y crimen, reducción de la po..."
931,d3284ee0-7209-46eb-b83c-645df6fd5af6,¿Cuál es la forma del parque de la Ciudadela e...,semicircular
932,7ba2a5a8-6ea4-4035-930a-9aab6e3c4644,¿Quién fue Joaquim Renart?,"un pintor, decorador y coleccionista, fundador..."


## Preguntas en las que todos los modelos han dado la misma respuesta, distinta al dataset

In [40]:
df_alt_ans

,id,question,answer,alt_answer,match
1,82ff0c00-c4a6-4677-af88-b9a05cce3b3f,"¿De dónde proviene el término ""célula"")","latín cellula, diminutivo de cella, ‘celda’",latín cellula,contains
3,cb1d5d94-ae86-4167-a8df-1de5a8a83038,¿Cómo se llaman las células que necesitan las ...,gliales,células gliales,contains
5,0e6afdc9-57a7-46b4-89cf-7923fe47d4ac,¿Cómo se logran resultados más decisivos respe...,mediante los estudios de microscopía electróni...,mediante los estudios de microscopía electróni...,contains
17,7e03463b-c0f8-45b9-8145-10839a3784b8,¿Dónde fueron descubiertos los restos históricos?,distrito de Aïn Shams,"distrito de Aïn Shams, al este de El Cairo",contains
19,75237723-d90d-4c14-8cbb-7aab4701a1b3,¿Cuál es la confederación con un menor porcent...,AFC,Concacaf,no
...,...,...,...,...,...
907,b27d726e-90d8-4a1e-83b6-c6c0f2372d34,¿Cómo se intentaba generar efectos lumínicos y...,gracias al aprovechamiento de las rugosidades ...,gracias al aprovechamiento de las rugosidades ...,contains
916,2ac11ff1-32bf-4d7c-a904-049027d6d14d,¿Cuándo se estrenó la primera película de cine...,1934,20 de mayo de 1897,no
917,5d544319-40c2-4b03-974a-071836a0766b,¿Cuándo se produjo el choque?,alrededor de las 15:40 [hora local],alrededor de las 15:40 [hora local,contains
928,91ade7b1-c915-4e03-ad44-5828d4421b9d,¿A cuánta distancia puede llegar el cañón de n...,cincuenta metros,más de cincuenta metros,contains


### Preguntas en las que todos los modelos han dado la misma respuesta, que no coincide en nada con la del dataset
(subconjunto de la tabla anterior)

In [41]:
df_alt_ans[df_alt_ans["match"]=="no"]

,id,question,answer,alt_answer,match
19,75237723-d90d-4c14-8cbb-7aab4701a1b3,¿Cuál es la confederación con un menor porcent...,AFC,Concacaf,no
24,7a0f986c-dd53-4917-96aa-8979bc8a7c74,¿Cuándo comenzaron las manifestaciones en el p...,las elecciones legislativas del pasado 15 de mayo,lunes,no
28,b7d6f4ea-126a-49c0-b244-af478a0356af,¿Qué puede causar que dejes tu equipo?,motivos profesionales,una mala racha,no
31,ff10c022-a4be-4db7-8a40-653e72c0538c,¿Cómo se declaraba el pueblo de Yugoslavia dur...,favorable a los Aliados,neutral,no
71,6103a299-6f4d-4bad-b229-11ebc95ee1f1,¿Cuándo se tradujo por primera vez al español ...,1777,1802 y 1807,no
91,474b9093-2478-4798-86e0-8f7493fbaaeb,¿Cuánto aumentaron los márgenes de las peras p...,un seis por ciento,62 por ciento,no
122,4380f375-42af-4529-b961-7e5cdd2197f4,¿Cuál era el nombre de Tallin tras la independ...,Tallinna,Reval,no
131,7630c71d-0c0e-4856-a2b2-d76d8020b5e2,¿Cómo se conoce también en castellano a las mu...,Damas del Imperio Invisible,Ku Klux Klan Women,no
155,7ceb2a51-f30e-48ef-aaf1-7a558f5a6dd5,¿Cómo se piensa que mejoraría la diversidad de...,disminuyendo el tamaño de las cuencas oceánicas,más abundante en aguas poco profundas,no
161,66f4324c-91ca-4426-8b7d-52a2ce65a9b6,¿Dónde se dice que los resultados de ADN sobre...,el libro titulado The Riddle of Anna Anderson,la prensa,no


In [42]:
df_alt_ans["match"].value_counts()

contains    103
no           38
Name: match, dtype: int64

## Preguntas en las que solo un modelo ha dado la misma respuesta del dataset

In [43]:
df_1h

,id,question,answer,bne_base_new,mbert,beto,model_dif
15,715c4249-f08b-455d-9da1-e973c58d72c2,¿Dónde se localizó el epicentro del seísmo?,"50 km bajo tierra en la prefectura de Ibaraki,...","50 km bajo tierra en la prefectura de Ibaraki,...",prefectura de Ibaraki,centro de Japón,bne_base_new
19,6b874d4b-1d54-44a3-8f47-ac15bdb0f17e,¿Qué porcentaje de la población mundial juega ...,4 %,4 %,alrededor del 4 %,alrededor del 4 %,bne_base_new
20,a9694543-cf58-4be1-9ca7-a025117b2911,¿Cuál es la probabilidad que tienen las orcas ...,extremadamente alta,el 37 y 50 % de las crías,extremadamente alta,el 37 y 50 %,mbert
31,f4395ed9-f089-4f39-b03d-09b660094f19,¿Cuál fue la posición que adoptó el gobierno y...,neutral,se declaró neutral,neutral,se declaró neutral,mbert
33,0a5e9251-abb0-437f-8fac-2f70952d6cc0,¿En qué período se sitúa la guerra de los huesos?,Gilded Age,durante la Gilded Age (Edad Dorada,durante la Gilded Age (Edad Dorada,Gilded Age,beto
...,...,...,...,...,...,...,...
910,1b9306c7-770e-4d7b-adef-eeed0c8d15ba,¿Qué rango militar tenía Augusto Pinochet?,Capitán General del Ejército (r) de Chile,ex presidente y Capitán General del Ejército,Capitán General del Ejército (r) de Chile,ex presidente y Capitán General,mbert
912,54b93118-02b4-4ac3-990f-e1c718c0100b,¿Dónde entra el amor?,el corazón humano,el corazón humano,corazón humano,corazón humano,bne_base_new
913,ff6f9cdf-245c-4a1e-a999-24301045c637,¿De qué se podría acusar a Israel según revela...,utilización de bombas de uranio en el Líbano,bombas anti búnker en la población de Khiam al...,utilizó bombas anti búnker en la población de ...,utilización de bombas de uranio en el Líbano,beto
919,77c9be4e-b200-4ec4-814a-02d5dd2e14f3,¿Cuándo finalizan las campañas de los que quie...,este miércoles (28,este miércoles (28,este miércoles,este miércoles,bne_base_new


In [44]:
df_1h["model_dif"].value_counts(normalize=True)

bne_base_new    0.369458
beto            0.339901
mbert           0.290640
Name: model_dif, dtype: float64